In [126]:
import tensorflow as tf
import pandas as pd
import re

In [144]:
file = open("Desktop/merged_data_with_got_ocr.csv", 'r')
df = pd.read_csv(file)

df['Institutional_Form'] = df['Institutional_Form'].str.lower()
df['Institutional_Form'] = df['Institutional_Form'].replace('cooperative purchasing agreement', 'cooperative purchasing arrangements')
df['Institutional_Form'] = df['Institutional_Form'].replace('transfer of authority', 'transfer of functions')
df['Institutional_Form'] = df['Institutional_Form'].replace('transfer of responsibility', 'transfer of functions')
df['Institutional_Form'] = df['Institutional_Form'].replace('joint facilities', 'joint facilities/co-location')
df['Institutional_Form'] = df['Institutional_Form'].replace('joint production', 'Joint Operations')
df['Institutional_Form'] = df['Institutional_Form'].replace('joint facilities/co-location and joint operations', 'joint facilities/co-location')
df['Institutional_Form'] = df['Institutional_Form'].replace('cooperative purchasing agreement and new joint entities', 'cooperative purchasing arrangements')
df['Institutional_Form'] = df['Institutional_Form'].replace('service contract and grants/other subsidies', 'service contract')
df['Institutional_Form'].value_counts()

Institutional_Form
service contract                       555
joint operations                       172
new joint entities                     120
resource sharing                        76
joint facilities/co-location            29
transfer of functions                   28
grants/other subsidies                  26
cooperative purchasing arrangements     15
Joint Operations                         5
coproduction                             5
other                                    2
Name: count, dtype: int64

In [146]:
def clean_text(text):
    # Remove newline-separated letters
    cleaned_text = re.sub(r'(\n[a-zA-Z])', '', text)
    # Remove single newlines or extraneous spaces
    cleaned_text = re.sub(r'\n+', ' ', cleaned_text).strip()
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    cleaned_text = re.sub(r'11.', ' ', cleaned_text).strip()
    cleaned_text = re.sub(r"\n\'", ' ', cleaned_text).strip()
    cleaned_text = re.sub(r'\n', ' ', cleaned_text).strip()
    ##Patterns found
    cleaned_text = re.sub(r'\n', ' ', cleaned_text).strip()
    cleaned_text = re.sub(r'\.{2,}\s{1,}', ' ', cleaned_text).strip()
    cleaned_text = re.sub(r'\.{2,}', ' ', cleaned_text).strip()
    cleaned_text = re.sub(r'\({2,}' , ' ', cleaned_text).strip()
    cleaned_text = re.sub(r'\){2,}'  , ' ', cleaned_text).strip()
    cleaned_text = re.sub(r'~{2,}', ' ', cleaned_text).strip()
    
    return cleaned_text

In [150]:
for i in range(len(df)):
    df.loc[i, 'got_ocr_text'] = clean_text(df['got_ocr_text'][i])


In [154]:
to_synth=["service contract","new joint entities","joint operations","resource sharing"]
filtered_df = df[df['Institutional_Form'].isin(to_synth)]
filtered_df = filtered_df.reset_index(drop=True)


In [222]:
import nlpaug.augmenter.word.context_word_embs as nawcwe

synth_df = pd.DataFrame({'synthetic_text': [], 'Institutional_Form': []})

for i in range(len(filtered_df)):
    aug = nawcwe.ContextualWordEmbsAug(model_path='bert-base-uncased',aug_p=0.4)
    if filtered_df['Institutional_Form'][i]=="service contract": num=1
    elif filtered_df['Institutional_Form'][i]=="resource sharing": num=3
    else: num=2
    augmented_texts = aug.augment(filtered_df["got_ocr_text"][i], n=num)
    augmented_texts = [x.replace(" ' ", "'") for x in augmented_texts]
    print("augmented document: ", i)
    for j in range(len(augmented_texts)):
        synth_df.loc[len(synth_df)] = [augmented_texts[j], filtered_df['Institutional_Form'][i]]

    

augmented document:  0
augmented document:  1
augmented document:  2
augmented document:  3
augmented document:  4
augmented document:  5
augmented document:  6
augmented document:  7
augmented document:  8
augmented document:  9
augmented document:  10
augmented document:  11
augmented document:  12
augmented document:  13
augmented document:  14
augmented document:  15
augmented document:  16
augmented document:  17
augmented document:  18
augmented document:  19
augmented document:  20
augmented document:  21
augmented document:  22
augmented document:  23
augmented document:  24
augmented document:  25
augmented document:  26
augmented document:  27
augmented document:  28
augmented document:  29
augmented document:  30
augmented document:  31
augmented document:  32
augmented document:  33
augmented document:  34
augmented document:  35
augmented document:  36
augmented document:  37
augmented document:  38
augmented document:  39
augmented document:  40
augmented document:  41
au

In [228]:
synth_df['Institutional_Form'].value_counts()

Institutional_Form
service contract      555
joint operations      344
new joint entities    240
resource sharing      228
Name: count, dtype: int64

In [230]:
synth_df.to_csv('word_embed_aug_contracts.csv', index=False) 